In [1]:
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os
import sys

Using TensorFlow backend.


In [2]:
classes = 7
img_row,img_col = 48,48
batch_size = 64
num_epoch = 100
num_train = 28709
num_val = 7178
train_data = "./fer2013/train"
validation_data ="./fer2013/validation"

In [3]:
## data Augumentation training data ##
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 30,
    shear_range=0.3,
    zoom_range=0.4,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode="nearest")

train_generator =  train_datagen.flow_from_directory(
        train_data,
        color_mode = 'grayscale',
        target_size = (img_row,img_col),
        batch_size  = batch_size,
        class_mode = 'categorical',
        shuffle = True ) 

validation_datagen = ImageDataGenerator(
    rescale=1./255)

vaildation_generator = validation_datagen.flow_from_directory(
    validation_data,color_mode = 'grayscale',
    target_size = (img_row,img_col),
    batch_size  = batch_size,
    class_mode = 'categorical',
    shuffle = True ) 

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [4]:
from keras.optimizers import Adam

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_row,img_col,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
model_info = model.fit_generator(train_generator,
                                 steps_per_epoch=num_train // batch_size,
                                 epochs=num_epoch,
                                 validation_data=vaildation_generator,
                                 validation_steps=num_val // batch_size)


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
448/448 [==============================] - 382s 854ms/step - loss: 1.8213 - acc: 0.2476 - val_loss: 1.8058 - val_acc: 0.2435
Epoch 2/100
448/448 [==============================] - 371s 829ms/step - loss: 1.7984 - acc: 0.2480 - val_loss: 1.8054 - val_acc: 0.2509
Epoch 3/100
448/448 [==============================] - 371s 827ms/step - loss: 1.7946 - acc: 0.2524 - val_loss: 1.7927 - val_acc: 0.2458
Epoch 4/100
448/448 [==============================] - 370s 827ms/step - loss: 1.7913 - acc: 0.2518 - val_loss: 1.7894 - val_acc: 0.2550
Epoch 5/100
448/448 [==============================] - 370s 827ms/step - loss: 1.7863 - acc: 0.2543 - val_loss: 1.7956 - val_acc: 0.2597
Epoch 6/100
448/448 [==============================] - 370s 827ms/step - loss: 1.7820 - acc: 0.2568 - val_loss: 1.7731 - val_acc: 0.2668
Epoch 7/100
448/448 [==============================] - 371s 827ms/step - loss: 1.7792 - acc: 0.2573 - val_loss: 1.7650 - val_acc: 

448/448 [==============================] - 372s 830ms/step - loss: 1.4389 - acc: 0.4467 - val_loss: 1.4949 - val_acc: 0.4850
Epoch 59/100
448/448 [==============================] - 371s 829ms/step - loss: 1.4345 - acc: 0.4452 - val_loss: 1.5188 - val_acc: 0.4828
Epoch 60/100
448/448 [==============================] - 370s 826ms/step - loss: 1.4269 - acc: 0.4505 - val_loss: 1.4979 - val_acc: 0.4926
Epoch 61/100
448/448 [==============================] - 371s 828ms/step - loss: 1.4322 - acc: 0.4476 - val_loss: 1.5207 - val_acc: 0.4872
Epoch 62/100
448/448 [==============================] - 384s 857ms/step - loss: 1.4218 - acc: 0.4533 - val_loss: 1.5535 - val_acc: 0.4916
Epoch 63/100
448/448 [==============================] - 371s 827ms/step - loss: 1.4265 - acc: 0.4468 - val_loss: 1.5279 - val_acc: 0.4896
Epoch 64/100
448/448 [==============================] - 370s 826ms/step - loss: 1.4140 - acc: 0.4566 - val_loss: 1.5075 - val_acc: 0.4929
Epoch 65/100
448/448 [=========================

In [8]:
 model.save_weights('face_emot_model.h5')

In [6]:
 emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


In [8]:
import cv2
import numpy as np
model.load_weights('face_emot_model.h5')

cap = cv2.VideoCapture(0)
while True:
# Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(720,480),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()